# Check if Open Street Map has good CA coverage

Would be nice to use `maxspeed` parameter.

Let's see where the waypoints and nodes are?

In [3]:
#!pip install osmium pyrosm ogr

In [ ]:
#https://pyrosm.readthedocs.io/en/latest/installation.html

In [4]:
from pyrosm.data import sources

# Print available source categories
sources.available.keys()

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


dict_keys(['africa', 'antarctica', 'asia', 'australia_oceania', 'central_america', 'europe', 'north_america', 'south_america', 'cities', 'subregions'])

In [6]:
from pyrosm import get_data, OSM

# Pyrosm comes with a couple of test datasets 
# that can be used straight away without
# downloading anything
fp = get_data("seattle")

# Initialize the OSM parser object
osm = OSM(fp)

# Read all drivable roads
# =======================
#drive_net = osm.get_network(network_type="driving")
#drive_net.plot()

Downloaded Protobuf data 'Seattle.osm.pbf' (36.29 MB) to:
'/tmp/pyrosm/Seattle.osm.pbf'


In [11]:
print(sources.subregions.usa.available)

['alabama', 'alaska', 'arizona', 'arkansas', 'colorado', 'connecticut', 'delaware', 'district_of_columbia', 'florida', 'georgia', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri', 'montana', 'nebraska', 'nevada', 'new_hampshire', 'new_jersey', 'new_mexico', 'new_york', 'north_carolina', 'north_dakota', 'northern_california', 'ohio', 'oklahoma', 'oregon', 'pennsylvania', 'puerto_rico', 'rhode_island', 'south_carolina', 'south_dakota', 'southern_california', 'tennessee', 'texas', 'utah', 'vermont', 'virginia', 'washington', 'west_virginia', 'wisconsin', 'wyoming']


In [ ]:
from pyrosm import OSM
osm_path ='../california-latest.osm.pbf' # downloaded from geofabrik
osm = OSM(osm_path)

In [ ]:
drive_net = osm.get_network(network_type="driving")
drive_net.plot()

In [12]:
import osmium as osm
import pandas as pd

In [13]:
#https://stackoverflow.com/questions/45771809/how-to-extract-and-visualize-data-from-osm-file-in-python
#http://andrewgaidus.com/Convert_OSM_Data/
#https://github.com/agaidus/Converting_OSM_Data/blob/master/Convert_OSM_Data.ipynb
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []

    def tag_inventory(self, elem, elem_type):
        for tag in elem.tags:
            self.osm_data.append([elem_type, 
                                   elem.id, 
                                   elem.version,
                                   elem.visible,
                                   pd.Timestamp(elem.timestamp),
                                   elem.uid,
                                   elem.user,
                                   elem.changeset,
                                   len(elem.tags),
                                   tag.k, 
                                   tag.v])

    def node(self, n):
        self.tag_inventory(n, "node")

    def way(self, w):
        self.tag_inventory(w, "way")

    def relation(self, r):
        self.tag_inventory(r, "relation")

In [14]:
osmhandler = OSMHandler()

#FILE = "../california-latest.osm.pbf"
#FILE = "/tmp/pyrosm/socal-latest.osm.pbf"
FILE = "/tmp/pyrosm/Seattle.osm.pbf"

# scan the input file and fills the handler list accordingly
osmhandler.apply_file(FILE)

In [15]:
#https://techoverflow.net/2021/04/25/minimal-example-how-to-read-osm-pbf-file-using-python-osmium/
print(f'Number of nodes: {osmhandler.node_count}')
print(f'Number of way: {osmhandler.way_count}')
print(f'Number of relations: {osmhandler.relation_count}')

AttributeError: 'OSMHandler' object has no attribute 'node_count'

In [44]:
class StreamHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.edge_and_nodes = [] 
    def way(self, w): 
        yield {'edge_id': w.id, 
               'nodes': [w.nodes[i].ref for i in range(len(w.nodes))]}
h = StreamHandler()
h.apply_file(FILE)
#for row in h.way(w): 
#    print(row) 

for i in h.way(w):
    print(i)

NameError: name 'w' is not defined

In [ ]:
# transform the list into a pandas DataFrame
data_colnames = ['type', 'id', 'version', 'visible', 'ts', 'uid',
                 'user', 'chgset', 'ntags', 'tagkey', 'tagvalue']
df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
#df_osm = tag_genome.sort_values(by=['type', 'id', 'ts'])

In [ ]:
df_osm.head()

In [ ]:
len(df_osm)

In [ ]:
# In terminal:
# wget -c http://download.geofabrik.de/north-america/us/california-latest.osm.pbf

# Tutorial: https://www.linuxbabe.com/linux-server/openstreetmap-tile-server-ubuntu-16-04

In [ ]:
# https://stackoverflow.com/questions/45771809/how-to-extract-and-visualize-data-from-osm-file-in-python
